In [1]:
import pickle

In [2]:
tokenized_stores = {'en_train': [], 'en_validation': [], 'en_test': [], 
                    'vi_train': [], 'vi_validation': [], 'vi_test': []}

In [3]:
for key in tokenized_stores:
    file_name = "D:/Statistical-Machine-Translation-master/data/" + str(key)[3:] + "." + str(key)[0:2]
    load = open(file_name, 'r', encoding='utf-8')
    sentences = load.read().split('\n')
    
    for sentence in sentences:
        token_store = sentence.split(' ')
        tokenized_stores[key].append(token_store)

In [8]:
print(tokenized_stores['en_train'][1])

['In', '4', 'minutes', ',', 'atmospheric', 'chemist', 'Rachel', 'Pike', 'provides', 'a', 'glimpse', 'of', 'the', 'massive', 'scientific', 'effort', 'behind', 'the', 'bold', 'headlines', 'on', 'climate', 'change', ',', 'with', 'her', 'team', '--', 'one', 'of', 'thousands', 'who', 'contributed', '--', 'taking', 'a', 'risky', 'flight', 'over', 'the', 'rainforest', 'in', 'pursuit', 'of', 'data', 'on', 'a', 'key', 'molecule', '.']


In [9]:
print(tokenized_stores['vi_train'][1])

['Trong', '4', 'phút', ',', 'chuyên', 'gia', 'hoá', 'học', 'khí', 'quyển', 'Rachel', 'Pike', 'giới', 'thiệu', 'sơ', 'lược', 'về', 'những', 'nỗ', 'lực', 'khoa', 'học', 'miệt', 'mài', 'đằng', 'sau', 'những', 'tiêu', 'đề', 'táo', 'bạo', 'về', 'biến', 'đổi', 'khí', 'hậu', ',', 'cùng', 'với', 'đoàn', 'nghiên', 'cứu', 'của', 'mình', '--', 'hàng', 'ngàn', 'người', 'đã', 'cống', 'hiến', 'cho', 'dự', 'án', 'này', '--', 'một', 'chuyến', 'bay', 'mạo', 'hiểm', 'qua', 'rừng', 'già', 'để', 'tìm', 'kiếm', 'thông', 'tin', 'về', 'một', 'phân', 'tử', 'then', 'chốt', '.']


In [11]:
train_size = len(tokenized_stores['vi_train'])
dev_size = len(tokenized_stores['vi_validation'])
test_size = len(tokenized_stores['vi_test'])
print(train_size)

133318


In [13]:
# making the vocabulary

en_words = {}
vi_words = {}

for key in tokenized_stores:
    if str(key)[0] == 'e':
        # creating en_words
        for sentence in tokenized_stores[key]:
            for word in sentence:
                if word in en_words:
                    en_words[word] += 1
                else:
                    en_words[word] = 1
    else:
        # creating vi_words
        for sentence in tokenized_stores[key]:
            for word in sentence:
                if word in vi_words:
                    vi_words[word] += 1
                else:
                    vi_words[word] = 1
                    
en_vocab = len(en_words)
vi_vocab = len(vi_words)
print("Number of Unique Words:")
print("> English:", str(en_vocab))
print("> Vietnamese:", str(vi_vocab))

Number of Unique Words:
> English: 54418
> Vietnamese: 25763


In [14]:
# creating the 't'
t = {}
# usage: t[('EN_word', 'VI_word')] = probability of EN_Word given VI_word
uniform = 1 / (en_vocab * vi_vocab)

In [15]:
n_iters = 0
max_iters = 25

fine_tune = 1
has_converged = False

while n_iters < max_iters and has_converged == False:
    has_converged = True
    max_change = -1

    n_iters += 1
    count = {}
    total = {}
    for index in range(train_size):
        s_total = {}
        for vi_word in tokenized_stores['vi_train'][index]:
            s_total[vi_word] = 0
            for en_word in tokenized_stores['en_train'][index]:
                if (vi_word, en_word) not in t:
                    t[(vi_word, en_word)] = uniform
                s_total[vi_word] += t[(vi_word, en_word)]

        for vi_word in tokenized_stores['vi_train'][index]:
            for en_word in tokenized_stores['en_train'][index]:
                if (vi_word, en_word) not in count:
                    count[(vi_word, en_word)] = 0
                count[(vi_word, en_word)] += (t[(vi_word, en_word)] / s_total[vi_word])

                if en_word not in total:
                    total[en_word] = 0
                total[en_word] += (t[(vi_word, en_word)] / s_total[vi_word])

    # estimating the probabilities

    if fine_tune == 0:
        updated = {}
        # train for all valid word pairs s.t count(vi_word, en_word) > 0
        for index in range(train_size):
            for en_word in tokenized_stores['en_train'][index]:
                for vi_word in tokenized_stores['vi_train'][index]:
                    if (vi_word, en_word) in updated:
                        continue
                    updated[(vi_word, en_word)] = 1
                    if abs(t[(vi_word, en_word)] - count[(vi_word, en_word)] / total[en_word]) > 0.01:
                        has_converged = False
                        max_change = max(max_change, abs(t[(vi_word, en_word)] - count[(vi_word, en_word)] / total[en_word]))
                    t[(vi_word, en_word)] = count[(vi_word, en_word)] / total[en_word]

    elif fine_tune == 1:
        # train it only for 1000 most frequent words in English and Hindi
        max_words = 1000
        n_en_words = 0
        updates = 0

        for en_word_tuples in sorted(en_words.items(), key = lambda k:(k[1], k[0]), reverse = True):
            en_word = en_word_tuples[0]
            n_en_words += 1
            if n_en_words > max_words:
                break
            n_vi_words = 0
            for vi_word_tuples in sorted(vi_words.items(), key = lambda k:(k[1], k[0]), reverse = True):
                vi_word = vi_word_tuples[0]
                n_vi_words += 1
                if n_vi_words > max_words:
                    break
                if (vi_word, en_word) not in count or en_word not in total:
                    continue
                    # assume in this case: t[(vi_word, en_word)] = uniform
                else:
                    if abs(t[(vi_word, en_word)] - count[(vi_word, en_word)] / total[en_word]) > 0.005:
                        has_converged = False
                        max_change = max(max_change, abs(t[(vi_word, en_word)] - count[(vi_word, en_word)] / total[en_word]))
                    t[(vi_word, en_word)] = count[(vi_word, en_word)] / total[en_word]

    print("Iteration " + str(n_iters) + " Completed, Maximum Change: " + str(max_change))


Iteration 1 Completed, Maximum Change: 0.19723033648703517
Iteration 2 Completed, Maximum Change: 0.3527098972236826
Iteration 3 Completed, Maximum Change: 0.22370977086941768
Iteration 4 Completed, Maximum Change: 0.1132108813751273
Iteration 5 Completed, Maximum Change: 0.07908404460848462
Iteration 6 Completed, Maximum Change: 0.056062286389514326
Iteration 7 Completed, Maximum Change: 0.04095588123126054
Iteration 8 Completed, Maximum Change: 0.03224587036832982
Iteration 9 Completed, Maximum Change: 0.026376635305706775
Iteration 10 Completed, Maximum Change: 0.021765884816976833
Iteration 11 Completed, Maximum Change: 0.018131465252190115
Iteration 12 Completed, Maximum Change: 0.01523931195885686
Iteration 13 Completed, Maximum Change: 0.012912288682408124
Iteration 14 Completed, Maximum Change: 0.011019827870231702
Iteration 15 Completed, Maximum Change: 0.009464452058678519
Iteration 16 Completed, Maximum Change: 0.008173907740638753
Iteration 17 Completed, Maximum Change: 0.0

In [16]:
# displaying the most confident translation pairs
limit = 40
for element in sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True):
    print(element)
    limit -= 1
    if limit <= 0:
        break

(('muốn', 'want'), 0.7294642574908756)
(('&quot;', '&quot;'), 0.6789230261909747)
(('biết', 'know'), 0.6269016245228031)
(('muốn', 'wanted'), 0.624979034147821)
(('Nhưng', 'But'), 0.6081539381806335)
(('10', '10'), 0.5940096178771291)
(('tôi', 'me'), 0.5906785947114103)
(('?', '?'), 0.5844060287159955)
(('năm', 'years'), 0.584177897113458)
(('khi', 'when'), 0.5814732203356352)
(('nghĩ', 'think'), 0.5714677610103552)
(('hiểu', 'understand'), 0.5706344209370805)
(('Nếu', 'If'), 0.5698436761487351)
(('20', '20'), 0.5681812570712279)
(('.', '.'), 0.5666718204194087)
(('Khi', 'When'), 0.5618504895822367)
(('cần', 'need'), 0.5610427257678787)
(('sẽ', '&apos;ll'), 0.5554436207744718)
(('30', '30'), 0.552651253768667)
(('không', '&apos;t'), 0.5521198231477583)
((',', ','), 0.550962898259224)
(('tháng', 'months'), 0.5500183170653205)
(('Và', 'And'), 0.5494389101071683)
(('ngày', 'day'), 0.54569340104206)
(('năm', 'year'), 0.5450873116024448)
(('không', 'not'), 0.5443756228444088)
(('bạn', 'you'

In [17]:
# saving the translation model
file = open("translation_model.pkl","wb")
pickle.dump(t, file)
file.close()

In [18]:
# using the model trained until convergence
# to use a saved model
model_name = "translation_model.pkl"
pickle_in = open(model_name,"rb")
t = pickle.load(pickle_in)

In [19]:
I = {}
for index in range(train_size):
    for vi_id in range(len(tokenized_stores['vi_train'][index])):
        length = len(tokenized_stores['vi_train'][index])
        if length not in I:
            I[length] = {} # maps the positional difference to a tuple: (sum of t's, count)
        for en_id in range(len(tokenized_stores['en_train'][index])):
            if (en_id - vi_id) not in I[length]:
                I[length][(en_id - vi_id)] = [t[(tokenized_stores['vi_train'][index][vi_id], tokenized_stores['en_train'][index][en_id])], 1]
            else:
                I[length][(en_id - vi_id)][0] += t[(tokenized_stores['vi_train'][index][vi_id], tokenized_stores['en_train'][index][en_id])]
                I[length][(en_id - vi_id)][1] += 1

In [20]:
# viewing the available sentence lengths encountered during training
sentence_lengths = []
for key in I.keys():
    if key not in sentence_lengths:
        sentence_lengths.append(key)
sentence_lengths.sort()
print(sentence_lengths)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 165, 166, 167, 168, 169, 170, 174, 177, 178, 179, 182, 183, 184, 191, 192, 197, 198, 200, 201, 203, 204, 208, 210, 211, 212, 221, 223, 224, 235, 236, 237, 243, 246, 247, 251, 279, 288, 298, 302, 306, 316, 322, 327, 332, 336, 344, 347, 371, 376, 386, 403, 423, 439, 464, 472, 481, 517, 559, 593, 614, 644, 85

In [21]:
# computing the alignment probabilities
# p[I][en_id - vi_id] = p(i | i', I)

p = {}
for key in I.keys():
    p[key] = {}
    sum_val = 0
    for diff in I[key].keys():
        p[key][diff] = I[key][diff][0] / I[key][diff][1]
        sum_val += p[key][diff]
    for diff in p[key].keys():
        p[key][diff] /= sum_val

In [22]:
for index in range(train_size):
    length_en = len(tokenized_stores['en_train'][index])
    length_vi = len(tokenized_stores['vi_train'][index])
    if length_en - length_vi > 10 and length_vi == 1:
        print("Length of English Sentence:", str(length_en))
        print("Length of Vietnamese Sentence:", str(length_vi))
        

In [23]:
# computing initial transitions
init = {}
for length in p:
    max_prob = -1
    max_jump = 0
    for key in p[length].keys():
        if p[length][key] > max_prob:
            max_prob = p[length][key]
            max_jump = key
    init[length] = max_jump

In [25]:
# computing the transition probabilities for English
bigrams = {}
unigrams = {}

# training on the train_set
def model(dataset_size, dataset_name):
    global bigrams
    global unigrams
    for index in range(dataset_size):
        token_A = ''
        for en_token in tokenized_stores[dataset_name][index]:
            if en_token not in unigrams:
                unigrams[en_token] = 1
            else:
                unigrams[en_token] += 1
            
            token_B = en_token
            if (token_A, token_B) not in bigrams:
                bigrams[(token_A, token_B)] = 1
            else:
                bigrams[(token_A, token_B)] += 1
            token_A = token_B

model(train_size, 'en_train')
model(dev_size, 'en_validation')

bigram_count = len(bigrams)
unigram_count = len(unigrams)
print("Number of Unique Bigrams:", bigram_count)
print("Number of Unique Unigrams:", unigram_count)

Number of Unique Bigrams: 580587
Number of Unique Unigrams: 54418


In [29]:
from itertools import permutations
import nltk

computed_sentences = []
total_BLEU = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 7: 0}
null_BLEU_count = 0

sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)

def find_translation(vi_token):
    for element in sorted_t:
        if element[0][0].lower() == vi_token:
            return element[0][1]
    return ""

def get_prob(seq):
    # bigram language model with laplace smoothing and backoff
    if len(seq) < 2:
        return 1
    score = 0
    token_A = ''
    for en_token in seq:
        token_B = en_token
        if (token_A, token_B) not in bigrams:
            if token_B not in unigrams:
                continue
            else:
                score += unigrams[token_B] / unigram_count
        else:
            base_token_count = 0
            if token_A in unigrams:
                base_token_count = unigrams[token_A]
            score += (bigrams[(token_A, token_B)] + 1) / (base_token_count + unigram_count)
        token_A = token_B
    return score

count = 0
for index in range(test_size):
    if len(tokenized_stores['vi_test'][index]) > 8 or len(tokenized_stores['vi_test'][index]) < 2:
        continue

    translated_words = []
    for vi_token in tokenized_stores['vi_test'][index]:
        translation = find_translation(vi_token)
        if translation != "":
            translated_words.append(translation)

    perm = permutations(translated_words)

    best_seq = translated_words
    best_prob = -1

    for seq in perm:
        prob = get_prob(seq)
        if prob > best_prob:
            best_prob = prob
            best_seq = seq

    BLEU_scores = []
    # Collecting BLEU_scores with various kinds of Smoothing
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['en_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method1))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['en_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method2))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['en_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method3))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['en_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['en_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method5))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['en_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method7))

    for key in total_BLEU.keys():
        if key == 7:
            consider = 5
        else: consider = key - 1
        total_BLEU[key] += BLEU_scores[consider]
    
    if BLEU_scores[0] == 0:
        null_BLEU_count += 1
    
    count += 1
    print("Sentence Index: ", str(count))
    print("Vietnamese Sentence:", str(tokenized_stores['vi_test'][index]))
    print("Reference English Sentence:", str(tokenized_stores['en_test'][index]))
    print("Translated Sentence:", str(best_seq))
    print("Translation BLEU Scores", str(BLEU_scores))
    print()
    
    computed_sentences.append([tokenized_stores['vi_test'][index], tokenized_stores['en_test'][index], best_seq, BLEU_scores])

tested = count

Sentence Index:  1
Vietnamese Sentence: ['Tôi', 'đã', 'bị', 'sốc', '.']
Reference English Sentence: ['I', 'was', 'so', 'shocked', '.']
Translated Sentence: ('already', 'device', '—', '.')
Translation BLEU Scores [0.06257106818159155, 0.24880469496253566, 0.12440234748126783, 0.04753271977233425, 0.07494014029189448, 0.10862101928796628]

Sentence Index:  2
Vietnamese Sentence: ['Nhưng', 'rất', 'nhiều', 'người', 'đã', 'chết', '.']
Reference English Sentence: ['But', 'many', 'die', '.']
Translated Sentence: ('very', 'already', 'die', 'people', '.', 'lot')
Translation BLEU Scores [0.048549177170732344, 0.2295748846661433, 0.09652434877402241, 0.044706344276931285, 0.10691671651659736, 0.133051885050407]

Sentence Index:  3
Vietnamese Sentence: ['Tôi', 'hoàn', 'toàn', 'tuyệt', 'vọng', '.']
Reference English Sentence: ['I', 'lost', 'all', 'hope', '.']
Translated Sentence: ('perfect', 'completely', 'wonderful', 'hope', '.')
Translation BLEU Scores [0.11362193664674995, 0.33980884896942454, 0

Sentence Index:  24
Vietnamese Sentence: ['Cám', 'ơn', 'rất', 'nhiều', '.']
Reference English Sentence: ['Thank', 'you', 'very', 'much', '.']
Translated Sentence: ('very', 'Thank', '.', 'lot')
Translation BLEU Scores [0.0823481567964712, 0.32744539334076506, 0.16372269667038256, 0.06255657725732221, 0.12490023381982417, 0.1600687290708062]

Sentence Index:  25
Vietnamese Sentence: ['Một', 'ngôi', 'trường', 'thật', 'sự', '.']
Reference English Sentence: ['A', 'real', 'school', '.']
Translated Sentence: ('house', 'environment', '.', 'truth', 'really')
Translation BLEU Scores [0.05372849659117709, 0.24028114141347542, 0.10682175159905853, 0.0456496931223525, 0.0898100418739418, 0.12114911497845275]

Sentence Index:  26
Vietnamese Sentence: ['Chúng', 'tôi', 'có', 'bị', 'theo', 'dõi', 'không', '?']
Reference English Sentence: ['Were', 'we', 'being', 'followed', '?']
Translated Sentence: ('?', 'follow', '&apos;t', 'device', '…', 'can', 'me')
Translation BLEU Scores [0.033031643180138064, 0.1

Sentence Index:  48
Vietnamese Sentence: ['Giống', 'như', 'thế', 'này', 'đây', '.']
Reference English Sentence: ['It', 'will', 'look', 'something', 'like', 'this', '.']
Translated Sentence: ('like', 'world', 'this', '.', 'here')
Translation BLEU Scores [0.08428828344718171, 0.2520807638479439, 0.1584873897212071, 0.08991803844670224, 0.12871616009783104, 0.15821989406845363]

Sentence Index:  49
Vietnamese Sentence: ['Và', 'đây', 'là', 'tôi', 'bây', 'giờ', '.']
Reference English Sentence: ['And', 'here', '&apos;s', 'me', 'today', '.']
Translated Sentence: ('here', 'me', '.', 'is', 'hours', 'Now')
Translation BLEU Scores [0.05372849659117709, 0.25406637407730737, 0.10682175159905853, 0.049475702907177795, 0.1283000598199168, 0.15467995414300717]

Sentence Index:  50
Vietnamese Sentence: ['Cảm', 'ơn', 'các', 'bạn', '.']
Reference English Sentence: ['Thank', 'you', '.']
Translated Sentence: ('companies', 'Thank', '.', 'you')
Translation BLEU Scores [0.10573712634405641, 0.4204482076268572

Sentence Index:  72
Vietnamese Sentence: ['Vì', 'thế', 'họ', 'giết', 'sư', 'tử', '.']
Reference English Sentence: ['So', 'they', 'kill', 'the', 'lions', '.']
Translated Sentence: ('death', 'scientists', 'they', 'world', '.', 'kill')
Translation BLEU Scores [0.05372849659117709, 0.25406637407730737, 0.10682175159905853, 0.049475702907177795, 0.1283000598199168, 0.15467995414300717]

Sentence Index:  73
Vietnamese Sentence: ['Nhưng', 'sư', 'tử', 'rất', 'thông', 'minh', '.']
Reference English Sentence: ['But', 'lions', 'are', 'very', 'clever', '.']
Translated Sentence: ('very', 'information', '.', 'scientists', 'death', 'smart')
Translation BLEU Scores [0.048549177170732344, 0.2295748846661433, 0.09652434877402241, 0.044706344276931285, 0.10691671651659736, 0.133051885050407]

Sentence Index:  74
Vietnamese Sentence: ['Và', 'tôi', 'lắp', 'đặt', 'mọi', 'thứ', '.']
Reference English Sentence: ['So', 'I', 'set', 'up', 'everything', '.']
Translated Sentence: ('me', 'second', 'everything', '—'

Sentence Index:  94
Vietnamese Sentence: ['Có', 'thể', 'mất', 'đến', 'hàng', 'tháng', '.']
Reference English Sentence: ['It', 'could', 'take', 'months', '.']
Translated Sentence: ('lost', 'can', 'until', '.', 'thousands', 'months')
Translation BLEU Scores [0.048549177170732344, 0.2295748846661433, 0.09652434877402241, 0.044706344276931285, 0.10691671651659736, 0.133051885050407]

Sentence Index:  95
Vietnamese Sentence: ['Nó', 'rất', 'tốn', 'thời', 'gian', '.']
Reference English Sentence: ['It', 'was', 'very', 'time-consuming', '.']
Translated Sentence: ('very', '—', '.', 'time', 'space')
Translation BLEU Scores [0.06389431042462725, 0.28574404296988, 0.1270331870386537, 0.05428693985879238, 0.11547005383792518, 0.1473464777541982]

Sentence Index:  96
Vietnamese Sentence: ['Một', 'đứa', 'bị', 'kẹt', 'trong', 'dòng', 'nước', '.']
Reference English Sentence: ['One', 'of', 'them', 'got', 'caught', 'up', 'in', 'the', 'water', '.']
Translated Sentence: ('device', 'line', 'inside', '—', '.'

In [30]:
# Results:
import statistics
print("Number of Samples Tested Upon: " + str(tested))
print()

print("Average BLEU Score using Various Smoothing Functions (considering all test samples)")
for key in total_BLEU:
    print("Method " + str(key) + ": " + str(total_BLEU[key] / tested))
print()
print("Average BLEU Score using Various Smoothing Functions (considering test samples with at-least one word overlap)")
for key in total_BLEU:
    print("Method " + str(key) + ": " + str(total_BLEU[key] / (tested - null_BLEU_count)))

Number of Samples Tested Upon: 111

Average BLEU Score using Various Smoothing Functions (considering all test samples)
Method 1: 0.05900411057601288
Method 2: 0.23700313830597766
Method 3: 0.11665190282683151
Method 4: 0.04867830067386414
Method 5: 0.09713658385108015
Method 7: 0.12299508107307175

Average BLEU Score using Various Smoothing Functions (considering test samples with at-least one word overlap)
Method 1: 0.06008675480676542
Method 2: 0.24135181974278458
Method 3: 0.11879230471356235
Method 4: 0.04957148050274238
Method 5: 0.0989189064905495
Method 7: 0.12525187155147674
